# Leje Data Scraper
#### Made By- Uday S.
#### For- Raphael G.

## I. Importing Libraries

In [1]:
#importing drivers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
#importing beautiful soup
from bs4 import BeautifulSoup
#importing the necessary libraries
import pandas as pd, numpy as np
#importing json package to read the data
import json
#importing os module
import os
from os.path import basename
#importing time
import time
#importing pathlib
import pathlib
#importing string
import string

## II. Defining Variable

In [2]:
#defining a function to find the position of a substring's nth instance in a string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [7]:
#Defining xpaths
Name = '//*[@id="auction-title-text"]/p'
Location = '//*[@id="info"]/div[1]/p/a'
Date_1 = '//*[@id="auction-title-dates"]/div[1]/div'
Val_1 = '//*[@id="auction-title-dates"]/div[1]/div'
Date_2 = '//*[@id="auction-title-dates"]/div[2]/div'
Val_2 = '//*[@id="auction-title-dates"]/div[2]/div'
Visits = '//*[@id="auction-views"]'
Commission= '//*[@id="winner-bid-value"]/small'
Description= '//*[@id="description"]'
Site_Process= '//*[@id="description"]/a'

## III. Scraping Page Links

In [3]:
# Using Selenium to define driver
driver = webdriver.Chrome(ChromeDriverManager().install())
#defining the base url
url= "https://www.leje.com.br/"
#opening the url
driver.get(url)
#defining variable
p_db=""
#getting the required element containing page links by class_name
p_element = driver.find_elements_by_class_name('auction-card-box ')

In [4]:
#Creating an empty DataFrame
url_df= pd.DataFrame()
#Creating an empty "url" column with the required rows
url_df["url"]= [None] * len(p_element)

In [5]:
#Using for loop to get the links of all the required pages
for i in range(len(p_element)):
    #getting the string having the link using get_attribute(), and assigning it to the column
    url_df["url"][i]=str(p_element[i].get_attribute("onclick"))
    #Cleaning the string to get the exact link for the page using find_nth function defined above
    url_df["url"][i]=url+url_df["url"][i][find_nth(url_df["url"][i],"index",1):find_nth(url_df["url"][i],"'",2)]

## IV. Pages' Data Scraping

### IV. A) Defining Df Columns

In [6]:
#Defining empty columns
url_df["Name"]=""
url_df["Location"]="" 
url_df["1st Square Date"]=""
url_df["Evaluation Value 1ª"]=""
url_df["2nd Square Date"]=""
url_df["Valuation Value 2nd"]=""
url_df["Views"]=""
url_df["Auctioneer commission"]=""
url_df["Useful area"]=""
url_df["Total area"]=""
url_df["Status"]=""
url_df["Garage"]=""
url_df["Floor"]=""
url_df["Municipal registration"]=""
url_df["Registration"]=""
url_df["Average condominium value"]=""
url_df["Average Annual IPTU value"]=""
url_df["Process number"]=""
url_df["Site Process"]=""
url_df["Payment methods"]=""
url_df["Tipo"]=""

### IV. B) Scraping Data Onto Respective Df Columns

In [8]:
#Defining driver
driver = webdriver.Chrome(ChromeDriverManager().install())
#Using for loop to get the necessary elements and assigning them to the url_df
for i in range(len(p_element)):
    #opening the url
    driver.get(url_df["url"][i])
    
    #getting the elements using xpath
    name_element= driver.find_element_by_xpath(Name)
    #using .text on the element to get the required text
    url_df["Name"][i]=name_element.text
    
    #getting the elements using xpath
    location_element= driver.find_element_by_xpath(Location)
    #using .text on the element to get the required text
    url_df["Location"][i]=location_element.text
    
    #getting the elements using xpath
    date_1_element= driver.find_element_by_xpath(Date_1)
    #using .text on the element to get the required text and using find_nth/find() to clean the text
    url_df["1st Square Date"][i]=date_1_element.text[find_nth(date_1_element.text," ",2)+1:date_1_element.text.find("\n")]
    
    #getting the elements using xpath
    val_1_element= driver.find_element_by_xpath(Val_1)
    #using .text on the element to get the required text and using find() to clean the text
    url_df["Evaluation Value 1ª"][i]=val_1_element.text[val_1_element.text.find("R$"):]
    
    #getting the elements using xpath
    date_2_element= driver.find_element_by_xpath(Date_2)
    #using .text on the element to get the required text and using find_nth/find() to clean the text
    url_df["2nd Square Date"][i]=date_2_element.text[find_nth(date_2_element.text," ",2)+1:date_2_element.text.find("\n")]
    
    #getting the elements using xpath
    val_2_element= driver.find_element_by_xpath(Val_2)
    #using .text on the element to get the required text and using find() to clean the text
    url_df["Valuation Value 2nd"][i]=val_2_element.text[val_2_element.text.find("R$"):]
    
    #getting the elements using xpath
    views_element= driver.find_element_by_xpath('//*[@id="auction-views"]')
    #using .text on the element to get the required text
    url_df["Views"][i]=views_element.text
    
    #getting the elements using xpath
    commission_element= driver.find_element_by_xpath(Commission)
    #using .text on the element to get the required text and using find() to clean the text
    url_df["Auctioneer commission"][i]=commission_element.text[commission_element.text.find("+")+1:]
    
    #getting the elements using xpath
    UtilArea_element= driver.find_element_by_xpath(Description)    
    #using .text on the element to get the required text
    Area_util=UtilArea_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "útil" in Area_util:
        Area_util=Area_util[Area_util.find("útil")+6:Area_util.find("\n")].strip(" ")
        if "•" in Area_util:
            Area_util=Area_util[:Area_util.find("•")].strip(" ")
    else:
        Area_util=""
    url_df["Useful area"][i]= Area_util   
    
    #getting the elements using xpath
    TotalArea_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Area_total=TotalArea_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "total" in Area_total:
        Area_total=Area_total[Area_total.find("total")+6:Area_total.find("\n")].strip(" ")
        if "•" in Area_total:
            Area_total=Area_total[:Area_total.find("•")].strip(" ")
    else:
        Area_total=""
    url_df["Total area"][i]= Area_total
    
    #getting the elements using xpath
    Status_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Status=Status_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Status" in Status:
        Status=Status[Status.find("Status")+7:Status.find("\n")].strip(" ")
        if "•" in Status:
            Status= Status[:Status.find("•")].strip(" ")
    else:
        Status=""
    url_df["Status"][i]=Status
    
    #getting the elements using xpath
    Garage_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Garage=Garage_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Garage" in Garage:
        Garage=Garage[Garage.find("Garage")+8:Garage.find("\n")].strip(" ")
        if "•" in Garage:
            Garage= Garage[:Garage.find("•")]
    else:
        Garage=""
    url_df["Garage"][i]= Garage
    
    #getting the elements using xpath
    Floor_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Floor=Floor_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Andar" in Floor:
        Floor=Floor[Floor.find("Andar")+7:Floor.find("\n")].strip(" ")
        if "•" in Floor:
            Floor= Floor[:Floor.find("•")]
    else:
        Floor=""
    url_df["Floor"][i]=Floor
    
    #getting the elements using xpath
    Municipal_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Municipal=Municipal_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Cadastro Municipal" in Municipal:
        Municipal=Municipal[Municipal.find("Cadastro Municipal")+19:].strip(" ")
        Municipal=Municipal[:Municipal.find(".")]
    else:
        Municipal=""
    url_df["Municipal registration"][i]=Municipal
    
    #getting the elements using xpath
    Registration_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Registration=Registration_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Matrícula" in Registration:
        Registration=Registration[Registration.find("Matrícula")+14:].strip(" ")
        Registration=Registration[:Registration.find("\n")]
    else:
        Registration=""
    url_df["Registration"][i]=Registration
    
    #getting the elements using xpath
    condominium_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    condominium=condominium_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Valor médio do Condomínio" in condominium:
        condominium=condominium[condominium.find("Valor médio do Condomínio")+27:].strip(" ")
        condominium=condominium[:condominium.find("\n")]
    else:
        condominium=""
    url_df["Average condominium value"][i]=condominium
    
    #getting the elements using xpath
    Iptu_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Iptu=Iptu_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Valor médio do IPTU" in Iptu:
        Iptu=Iptu[Iptu.find("Valor médio do IPTU")+20:].strip(" ")
        Iptu=Iptu[:Iptu.find("\n")]
    else:
        Iptu=""
    url_df["Average Annual IPTU value"][i]=Iptu
    
    #getting the elements using xpath
    process_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    process=process_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Processo nº:" in process:
        process=process[process.find("Processo nº:")+15:].strip(" ")
        process=process[:process.find("\n")]
    else:
        process=""
    url_df["Process number"][i]=process
    
    #getting the elements using xpath
    site_element= driver.find_element_by_xpath(Site_Process)
    #using .get_attribute() on the element to get the required text 
    site=site_element.get_attribute("href")
    url_df["Site Process"][i]=site
    
    #getting the elements using xpath
    Payment_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Payment=Payment_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Formas de Pagamento" in Payment:
        Payment=Payment[Payment.find("Formas de Pagamento")+20:].strip(" ")
        Payment=Payment[:Payment.find("\n")]
    else:
        Payment=""
    url_df["Payment methods"][i]=Payment
    
    #getting the elements using xpath
    Tipo_element= driver.find_element_by_xpath(Description)
    #using .text on the element to get the required text
    Tipo=Tipo_element.text
    #using if/else statement to clean the data appropriately based on the position of the data in the Descroption Tab
    if "Tipo:" in Tipo:
        Tipo=Tipo[Tipo.find("Tipo:")+6:Tipo.find("\n")].strip(" ")
        if "•" in Tipo:
            Tipo=Tipo[:Tipo.find("•")].strip(" ")
    else:
        Tipo=""
    url_df["Tipo"][i]=Tipo
    print(i)

### VI. Downloading The Data

In [9]:
#Downloading the url_df to an excel file
url_df.to_excel("Leje.xlsx",index=False)